Design idea:

A graph can represent the pipeline. 

1.  Build the pipeline naively
2.  Rearrange the order of the pipeline by the distance to `root` node

In [1]:
import numpy as np
import pandas as pd

from hinge import Hinge, error_on_split, error_on_split_reg
from interaction import Interaction

from sklearn import datasets
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

# using svm as per scikit-feature repo
from sklearn.decomposition import FactorAnalysis # we shall use factor analysis to fix the size of final modelling ds.
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# for sampling
import random # use random.choice? and random.sample for interactions

# use networkx to keep track of changes to our data, so we can recreate things...
import networkx as nx
import matplotlib.pyplot as plt
from graph_utils import *
import itertools
%matplotlib inline

In [2]:
import pandas as pd # for making sure and exploring created datasets...
X_df = pd.read_csv("trees.csv")
X_df.head()

,Girth,Height,Volume
0,8.3,70.0,10.3
1,8.6,65.0,10.3
2,8.8,63.0,10.2
3,10.5,72.0,16.4
4,10.7,81.0,18.8


In [3]:
X_df.columns

Index(['Girth', 'Height', 'Volume'], dtype='object')

In [4]:
y = X_df['Volume']

In [5]:
X_df.drop("Volume", axis=1, inplace=True)

In [6]:
def eval_pipeline(additional_feats=[], X=X_df, y=y, verbose=True):
    #print(additional_feats)
    
    pipeline = additional_feats[:]
    
    total_vars = len(pipeline) + len(X_df.columns)
    dim_factor = max(min(total_vars, 20), len(X_df.columns))
    
    pipeline.append(('factor analysis', FactorAnalysis(dim_factor)))
    pipeline.append(('linear reg', LinearRegression()))
    model = Pipeline(pipeline[:])

    # split data into 10 folds
    kfold = KFold(n_splits=10, shuffle=True)
    scoring = 'neg_mean_squared_error'
    scoring = 'r2'
    #scoring = 'neg_mean_absolute_error'
    results = cross_val_score(model, X, y, cv=kfold, scoring=scoring)
    if verbose:
        print("{}: {}".format(scoring, results.mean()))
    return results.mean()



In [7]:
# baseline solution - vanilla pipeline
eval_pipeline(X=X_df)

r2: 0.7644716475570899


0.76447164755708985

Feature Search
=======

We have 4 possible situations:

1. Grow - split : assumption is we always grow 2
2. Grow - interaction
3. Remove - split : assumption is we always destroy both
4. Remove - interaction

Proposal Distributions
------------------------

Assume each action has equal probability and is uniform.

Then each proposal is simply:

$$P(x^*|x^{(t)}) = \frac{1}{\text{num possible actions}}$$


------

In this setting there is no need for RJMCMC (yet) as the transformations are parameter free.

What this means is we can do a straight MH

In [8]:
"""
Graph details...

hinge parents will have attribute name hinge_children with value equal to the node name of the children
interaction parents will have attribute name interaction_children with value equal to list of all nodes that are children
"""

# generate the base graph, where we have a the root node and all the base features...
G=nx.DiGraph()
G.add_node("root")

for col in X_df.columns:
    # we can add node attributes, eg
    #G.add_node(col, attribute='here')
    G.add_node(col)
    G.add_edge("root", col)

# add attribute to node base_0
#G = set_graph_node_attributes(G, 'base_0', {'hinge_children': ['a', 'b']})




In [9]:
graph_to_dict(G)

{'edges': [['root', 'Girth', {}], ['root', 'Height', {}]],
 'nodes': [['Girth', {}], ['root', {}], ['Height', {}]]}

In [10]:
def get_all_nodes_by_attribute(G, attri_key):
    g_nodes_attr = graph_to_dict(G)['nodes']
    g_nodes = [node for node, attr in g_nodes_attr if attri_key in attr.keys()]
    return g_nodes

def get_graph_attributes_by_key(G, attri_key):
    g_nodes_attr = graph_to_dict(G)['nodes']
    attr_values = [attri.get(attri_key, None) for _, attri in g_nodes_attr]
    attr_values = [x for x in attr_values if x is not None]
    return attr_values

def get_all_node_attributes(G, node):
    return G[node]

In [11]:
def set_graph_node_attributes(G1, node, attri_dict):
    """
    G is a networkx graph...
    """
    G = G1.copy()
    
    #dict_items = attri_dict.items()    
    for attri_name, attri_value in attri_dict.items():
        nx.set_node_attributes(G, attri_name, {node: attri_value})
    return G.copy()

In [12]:
"""
Migrate this to another module in the future

Details
-------

### hinge

**parent node**

Will have details of the children in "hinge_children". 
That is...

```
Hinge(mask=parent)
```

A parent node will create two children

pos_name = "{}_poshinge".format(remove_node_name)
neg_name = "{}_neghinge".format(remove_node_name)

### interaction

**child node**

Will have details of the parent in "interaction_parent"
That is...

```
Interaction(['interaction1', 'interaction2'])
```

the node created will be the name

"{}_{}".format(inter1, inter2)

"""

def get_remove_hinge_candidates(G):
    # get all nodes with attribute "hinge_children"
    # where their totol children is 2 or less.    
    hinge_parents = get_all_nodes_by_attribute(G, 'hinge_children')
    
    # now check that all hinge parents have 2 or less children...
    hinge_parents = [parent for parent in hinge_parents if len(nx.descendants(G, parent)) <= 2]
    return hinge_parents

def get_grow_hinge_candidates(G):
    # these are all nodes who do not have a hinge children attribute. 
    hinge_parents = get_all_nodes_by_attribute(G, 'hinge_children')
    hinge_candidates = [node for node in G.nodes() if node not in hinge_parents and node != "root"]
    return hinge_candidates

def get_remove_interact_candidates(G):
    # get all nodes with attribute "interaction_parent"
    # where they have no children
    interaction_nodes = get_all_nodes_by_attribute(G, 'interaction_parent')
    # now check that all hinge parents have 2 or less children...
    nodes = [node for node in interaction_nodes if len(nx.descendants(G, node)) == 0]
    return nodes
    
def get_grow_interact_candidates(G):
    # get all pairwise nodes which are not currently a pair?
    try:
        all_nodes = G.nodes()
    except:
        all_nodes = G['nodes']
    all_nodes = [x for x in all_nodes if x != "root"]
    
    # generate all pairs...
    pairwise = itertools.combinations(all_nodes, 2)
    itself = zip(*[all_nodes, all_nodes])
    all_pairs = list(pairwise) + list(itself)
    
    seen_pairs = get_graph_attributes_by_key(G, 'hinge_children')
    seen_pairs = [set([x,y]) for x,y in seen_pairs]
    # remove "seen" pairs.
    filter_pairs = [set([x,y]) for x,y in all_pairs if set([x,y]) not in seen_pairs]
    return filter_pairs

In [13]:
# begin trying stuff
# start simulations!
def spawn_possible_actions(G):
    actions = []
    if get_remove_interact_candidates(G):
        actions.append(("remove", "interact"))
    if get_remove_hinge_candidates(G):
        actions.append(("remove", "hinge"))
    if get_grow_hinge_candidates(G):
        actions.append(("grow", "hinge"))
    if get_grow_interact_candidates(G):
        actions.append(("grow", "interact"))
    return actions

spawn_possible_actions(G)

[('grow', 'hinge'), ('grow', 'interact')]

In [14]:
def spawn_action_candidates(G, action, transform):
    if action == 'grow':
        if transform == 'hinge':
            return get_grow_hinge_candidates(G)
        elif transform == 'interact':
            return get_grow_interact_candidates(G)
        else:
            raise Exception("invalid transform function")
    if action == 'remove':
        if transform == 'hinge':
            return get_remove_hinge_candidates(G)
        elif transform == 'interact':
            return get_remove_interact_candidates(G)
        else:
            raise Exception("invalid transform function")
    else:
        raise Exception("invalid grow function")

In [15]:
def spawn_new_model(G):
    # to spawn a model, spawn possible actions:    
    all_actions = spawn_possible_actions(G)
    action, transform = random.choice(all_actions)
    # this just returns the function that is randomly chosen.
    candidates = spawn_action_candidates(G, action, transform)
    selected_node = random.choice(candidates)
    return {'action': action, 
            'transform': transform,
            'selected_node': selected_node}
    

In [16]:
def spawn_new_pipeline(G, mode="classification"):
    """
    Take in a networkx graph as a dictionary object
    
    We will convert a graph to pipeline objects,
    and then reorder based on distance to root node. 
    
    Usage for shortest path: 
    print(nx.shortest_path_length(G,source=0,target=4))
    
    hopefully this works...
    """
    # just randomly add...pipeline transformations   
    g_nodes = graph_to_dict(G)['nodes']
    
    # add all hinge nodes...
    if mode == "classification":
        hinge_info = [("hinge_{}".format(node), Hinge(mask=node, psplit=error_on_split), node,
                  nx.shortest_path_length(G,source="root",target=node)) 
                  for node, attr in g_nodes if 'hinge_children' in attr.keys()]
    else: 
        hinge_info = [("hinge_{}".format(node), Hinge(mask=node, psplit=error_on_split_reg), node,
                  nx.shortest_path_length(G,source="root",target=node)) 
                  for node, attr in g_nodes if 'hinge_children' in attr.keys()]
    
    
    # add all interaction nodes...
    # length + 1 as interaction thing is at parent level? when we consider things
    # like removal
    interact_info = [("interact_{}".format('_'.join(attr['interaction_parent'])), 
                      Interaction(attr['interaction_parent']), 
                      node, 
                      nx.shortest_path_length(G,source="root",target=node)+1)
                     for node, attr in g_nodes if 'interaction_parent' in attr.keys()]
    
    
    # join all together
    pipeline = hinge_info + interact_info
    
    # sort the pipeline by location of node to root...
    pipeline.sort(key=lambda x: x[3])
    #print(pipeline)
    # remove last element in pipeline..
    pipeline = [(x, y) for x,y,_,_ in pipeline]
    return pipeline

In [17]:
### spawn one pass...
new_model = spawn_new_model(G)
print("Chosen action/transform: {} {}".format(new_model['action'], new_model['transform']))
print("selected node(s): {}".format(new_model['selected_node']))

Chosen action/transform: grow hinge
selected node(s): Height


In [35]:
def _proposal(G, action, transform):
    """
    G is current graph of model.
    
    return proportional probabilities related to proposal distributions,
    both the current proposal and the "proposed" proposal
    
    x*|xt is proposal prime
    xt|x* is proposal cur
    
    available transforms:
    if get_remove_interact_candidates(G):
    if get_remove_hinge_candidates(G):
    if get_grow_hinge_candidates(G):
    if get_grow_interact_candidates(G):

    use this: simulate_new_graph

    """
    if action == "remove" and transform == "interact":
        all_pos_parents = get_remove_interact_candidates(G)
        proposal_prime = 1.0/len(all_pos_parents)
        
        # simulate removing a pair...
        all_interact_nodes = graph_to_dict(G.copy())['nodes']
        all_interact_nodes = [node for node, attr in all_interact_nodes if 'interaction_parent' in attr.keys()]
        
        G_prime = simulate_new_graph(G.copy(), {
            'action': action, 'transform': transform, 'selected_node': all_interact_nodes[0]
        })
        G_prime = dict_to_graph(G_prime)
        
        proposal_cur = 1.0/len(get_grow_interact_candidates(G_prime))
        
    if action == "remove" and transform == "hinge":
        all_pos_parents = get_remove_hinge_candidates(G)
        proposal_prime = 1.0/len(all_pos_parents)
        
        # simulate removing a pair
        #G_prime = G.copy()
        G_prime = simulate_new_graph(G.copy(), {
            'action': action, 'transform': transform, 'selected_node': all_pos_parents[0]
        })
        G_prime = dict_to_graph(G_prime)
        proposal_cur = 1.0/len(get_grow_hinge_candidates(G_prime))
        
                
    if action == "grow" and transform == "hinge":
        all_pos_candidates = get_grow_hinge_candidates(G)
        proposal_prime = 1.0/len(all_pos_candidates)
        
        # simulate adding a random proposal
        G_prime = simulate_new_graph(G.copy(), {
            'action': action, 'transform': transform, 'selected_node': all_pos_candidates[0]
        })
        G_prime = dict_to_graph(G_prime)
        proposal_cur = 1.0/len(get_remove_hinge_candidates(G_prime))
        
    if action == "grow" and transform == "interact":
        all_pos_candidates = get_grow_interact_candidates(G)
        proposal_prime = 1.0/len(all_pos_candidates)

        # simulate adding a random proposal
        G_prime = G.copy()
        
        # interaction
        G_prime = simulate_new_graph(G.copy(), {
            'action': action, 'transform': transform, 'selected_node': all_pos_candidates[0]
        })
        G_prime = dict_to_graph(G_prime)
        proposal_cur = 1.0/len(get_remove_interact_candidates(G_prime))
    
    # add fudge scaling factor
    proposal_cur = min(proposal_cur, 0.25)
    proposal_prime = min(proposal_prime, 0.25)
    
    return proposal_cur, proposal_prime
    
def proposal(G, action, transform):
    try: 
        proposal_cur, proposal_prime = _proposal(G, action, transform)
        return proposal_cur, proposal_prime
    except ZeroDivisionError as err:
        print("Warning: Move appears to be invalid")
        return 1.0, 1.0

In [36]:
def _acceptance(proba_cur, proba_prime, proposal_cur, proposal_prime):
    accept = (proba_prime/proba_cur) * (proposal_cur/proposal_prime)
    return min(1.0, accept)

def propose_iter(G_dict, mode="regression"):
    """
    Takes in list of graphs as dict objects    
    """
    last_model = G_dict.copy()
    #print(last_model )
    curr_G = dict_to_graph(last_model)
    propose_model = spawn_new_model(dict_to_graph(last_model)) # provide the action/transform pair
    proposed_graph = simulate_new_graph(curr_G, propose_model)
    curr_pipeline = spawn_new_pipeline(curr_G, mode=mode) # maybe rename to spawn_pipeline
    propose_pipeline = spawn_new_pipeline(dict_to_graph(proposed_graph), mode=mode)
    
    #print(propose_pipeline)
    #print(curr_pipeline)
    proba_cur = eval_pipeline(additional_feats=curr_pipeline, X=X_df, verbose=False)
    #print(propose_pipeline)
    proba_prime = eval_pipeline(additional_feats=propose_pipeline, X=X_df, verbose=False)
    
    proposal_cur, proposal_prime = proposal(curr_G, propose_model['action'], propose_model['transform'])
    
    result = {'acceptance_proba': _acceptance(proba_cur, proba_prime, proposal_cur, proposal_prime)}
    
    result_new = result.copy()
    result_new.update(propose_model)
    result['proposed_graph'] = curr_G.copy
    return result_new

# if proposal is accepted...update G and transform list here...

In [37]:
def simulate_new_graph(G_new, proposed):
    """
    G_new is a graph
    proposed is a dict like this: {'action': 'grow', 'selected_node': 'base_12', 'transform': 'hinge'}
    
    return a graph as a dict obj
    
    """
    #G_new = dict_to_graph(G_list[-1].copy())    
    
    # now in G, we add or remove as needed...
    if proposed['action'] == 'grow':
        if proposed['transform'] == 'hinge':
            #print(proposed['selected_node'])
            pos_name = "{}_poshinge".format(proposed['selected_node'])
            neg_name = "{}_neghinge".format(proposed['selected_node'])
            G_new.add_node(pos_name)
            G_new.add_node(neg_name)
            G_new.add_edge(proposed['selected_node'], pos_name)
            G_new.add_edge(proposed['selected_node'], neg_name)
            
            # add attributes, which is to the parent node.
            attributes = get_all_node_attributes(G_new, proposed['selected_node']).copy()
            attributes['hinge_children'] = [pos_name, neg_name]
            G_new = set_graph_node_attributes(G_new, proposed['selected_node'], attributes)
            
        elif proposed['transform'] == 'interact':
            inter_nodes = list(proposed['selected_node'])
            #print(inter_nodes)
            if len(inter_nodes) == 1:
                inter_nodes = inter_nodes[:] + inter_nodes[:]
            interact_name = "{}_{}".format(inter_nodes[0], inter_nodes[1])
            G_new.add_node(interact_name)
            G_new.add_edge(inter_nodes[0], interact_name)
            G_new.add_edge(inter_nodes[1], interact_name)
            
            # add attributes, which is on the child node
            attributes = get_all_node_attributes(G_new, interact_name).copy()
            attributes['interaction_parent'] = [inter_nodes[0], inter_nodes[1]]
            G_new = set_graph_node_attributes(G_new, interact_name, attributes)
        else:
            raise Exception("action/transform pair: {} {} appears to be invalid".format(proposed['action'], proposed['transform']))
    elif proposed['action'] == 'remove':        
        if proposed['transform'] == 'hinge':
            #print("\tremove all child nodes of node {}".format(proposed['selected_node']))            
            all_children = nx.descendants(G, proposed['selected_node'])
            for node in all_children:
                G_new.remove_node(node)
        elif proposed['transform'] == 'interact':
            #print("\tremove interact node {}".format(proposed['selected_node']))
            G_new.remove_node(proposed['selected_node'])
        else:
            raise Exception("action/transform pair: {} {} appears to be invalid".format(proposed['action'], proposed['transform']))    
    else:
        raise Exception("action/transform pair: {} {} appears to be invalid".format(proposed['action'], proposed['transform']))
    return graph_to_dict(G_new)

In [38]:
def create_iter(G_list, proposed):
    G_last = dict_to_graph(G_list[-1].copy())    
    G_new_dict = simulate_new_graph(G_last, proposed)
    
    # add things...
    G_new_list = G_list[:]
    G_new_list.append(G_new_dict)
    return G_new_list[:]

In [39]:
def mh_iter(G_list):
    u = np.random.uniform()    
    proposed = propose_iter(G_list[-1])
    
    # make u =0 .
    print(proposed['acceptance_proba'])
    if u < proposed['acceptance_proba']:
        #print(proposed)
        return create_iter(G_list, proposed)
    else:
        print("repeat previous\n\t")
        #print(proposed)
        # repeat the previous one...
        return G_list[:]


In [40]:
# generate the base graph, where we have a the root node and all the base features...
G=nx.DiGraph()
G.add_node("root")

for col in X_df.columns:
    # we can add node attributes, eg
    #G.add_node(col, attribute='here')
    G.add_node(col)
    G.add_edge("root", col)

In [41]:
iters = 10

G_list = [graph_to_dict(G)]

for i in range(iters):
    print(i)
    """
    try:
        G_list_temp = mh_iter(G_list[:])
        G_list = G_list_temp[:]
    except:
        print("mh iter failed")
        pass
    """
    G_list_temp = mh_iter(G_list[:])
    G_list = G_list_temp[:]

    if i % 1 == 0:
        pipeline = G_list[-1]
        check_pipeline = spawn_new_pipeline(dict_to_graph(pipeline))
        human_readable_pipeline = [x for x, y in check_pipeline]
        #print("Evaluating pipeline: {}".format(spawn_pipeline))
        perf = eval_pipeline(additional_feats=check_pipeline, X=X_df, verbose=False)
        print("Performance at pipeline: {} is \n\t{}\n".format(human_readable_pipeline, perf))

0
-15.4220013974
repeat previous
	
Performance at pipeline: [] is 
	0.7523083814048224

1
0.937194324947
Performance at pipeline: ['hinge_Girth'] is 
	0.4978047560169415

2
1.0
Performance at pipeline: ['hinge_Girth'] is 
	0.7788812033908351

3
1.0
Performance at pipeline: ['hinge_Girth', 'interact_Height_Height'] is 
	0.7827515416996759

4
1.0
Performance at pipeline: ['hinge_Girth'] is 
	0.3434789682760645

5
1.0
Performance at pipeline: ['hinge_Girth', 'interact_Height_Girth_poshinge'] is 
	0.8476178836785427

6
1.0
Performance at pipeline: ['hinge_Girth', 'interact_Height_Girth_poshinge', 'interact_Girth_neghinge_Girth'] is 
	0.5725472746571394

7
0.483367799181
repeat previous
	
Performance at pipeline: ['hinge_Girth', 'interact_Height_Girth_poshinge', 'interact_Girth_neghinge_Girth'] is 
	-0.16837283988044885

8
1.0
Performance at pipeline: ['hinge_Girth', 'interact_Height_Girth_poshinge', 'interact_Girth_neghinge_Girth', 'interact_Height_Girth_poshinge_Height'] is 
	0.4801785961

In [44]:
# proposed model in R is...

eval_pipeline(additional_feats=[
    ('girth hinge', Hinge(mask='Girth', psplit=error_on_split_reg)),
    ('height hinge', Hinge(mask='Height', psplit=error_on_split_reg))
], X=X_df, verbose=False)

0.71117791046422352

In [45]:
# proposed model in R is...

eval_pipeline(additional_feats=[], X=X_df, verbose=False)

0.23377071992841381